# TC 5033
## Deep Learning
## Fully Connected Deep Neural Networks

#### Activity 1b: Implementing a Fully Connected Network for Kaggle ASL Dataset

- Objective

The aim of this part of the activity is to apply your understanding of Fully Connected Networks by implementing a multilayer network for the [Kaggle ASL (American Sign Language) dataset](https://www.kaggle.com/datasets/grassknoted/asl-alphabet). While you have been provided with a complete solution for a Fully Connected Network using Numpy for the MNIST dataset, you are encouraged to try to come up with the solution.

- Instructions

    This activity requires submission in teams of 3 or 4 members. Submissions from smaller or larger teams will not be accepted unless prior approval has been granted (only due to exceptional circumstances). While teamwork is encouraged, each member is expected to contribute individually to the assignment. The final submission should feature the best arguments and solutions from each team member. Only one person per team needs to submit the completed work, but it is imperative that the names of all team members are listed in a Markdown cell at the very beginning of the notebook (either the first or second cell). Failure to include all team member names will result in the grade being awarded solely to the individual who submitted the assignment, with zero points given to other team members (no exceptions will be made to this rule).

    Load and Preprocess Data: You are provided a starter code to load the data. Be sure to understand the code.

    Review MNIST Notebook (Optional): Before diving into this activity, you have the option to revisit the MNIST example to refresh your understanding of how to build a Fully Connected Network using Numpy.

    Start Fresh: Although you can refer to the MNIST solution at any point, try to implement the network for the ASL dataset on your own. This will reinforce your learning and understanding of the architecture and mathematics involved.

    Implement Forward and Backward Pass: Write the code to perform the forward and backward passes, keeping in mind the specific challenges and characteristics of the ASL dataset.
    
     Design the Network: Create the architecture of the Fully Connected Network tailored for the ASL dataset. Choose the number of hidden layers, neurons, and hyperparameters judiciously.

    Train the Model: Execute the training loop, ensuring to track performance metrics such as loss and accuracy.

    Analyze and Document: Use Markdown cells to document in detail the choices you made in terms of architecture and hyperparameters, you may use figures, equations, etc to aid in your explanations. Include any metrics that help justify these choices and discuss the model's performance.  

- Evaluation Criteria

    - Code Readability and Comments
    - Appropriateness of chosen architecture and hyperparameters for the ASL dataset
    - Performance of the model on the ASL dataset (at least 70% acc)
    - Quality of Markdown documentation

- Submission

Submit this Jupyter Notebook in canvas with your complete solution, ensuring your code is well-commented and includes Markdown cells that explain your design choices, results, and any challenges you encountered.


In [37]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [38]:
%pip install --upgrade --force-reinstall numpy

  Using cached numpy-2.4.2-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
Using cached numpy-2.4.2-cp312-cp312-win_amd64.whl (12.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.4.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.4.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.2 which is incompatible.
scipy 1.13.1 requires numpy<2.3,>=1.22.4, but you have numpy 2.4.2 which is incompatible.


In [39]:
%pip install --upgrade --force-reinstall numpy

import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import os

%load_ext autoreload
%autoreload 2
#################################
%matplotlib inline


[autoreload of numpy.core failed: Traceback (most recent call last):
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    if not append_obj(module, old_objects, name, obj):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\IPython\extensions\autoreload.py", line 423, in append_obj
    in_module = hasattr(obj, "__module__") and obj.__module__ == module.__name__
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\numpy\core\defchararray.py", line 2, in __getattr__
    from numpy._core import defchararray
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\numpy\_core\defchararray.py", line 23, in <modu

  Using cached numpy-2.4.2-cp312-cp312-win_amd64.whl.metadata (6.6 kB)
Using cached numpy-2.4.2-cp312-cp312-win_amd64.whl (12.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.4.2
    Uninstalling numpy-2.4.2:
      Successfully uninstalled numpy-2.4.2
Note: you may need to restart the kernel to use updated packages.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.4.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.4.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.2 which is incompatible.
scipy 1.13.1 requires numpy<2.3,>=1.22.4, but you have numpy 2.4.2 which is incompatible.


In [ ]:
# Define the path to your data folder (update if needed)
DATA_PATH = '\asl_data'  # or the correct path to your data

train_df = pd.read_csv(os.path.join(DATA_PATH, 'sign_mnist_train.csv'))
valid_df = pd.read_csv(os.path.join(DATA_PATH, 'sign_mnist_valid.csv'))

[autoreload of numpy.core failed: Traceback (most recent call last):
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    if not append_obj(module, old_objects, name, obj):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\IPython\extensions\autoreload.py", line 423, in append_obj
    in_module = hasattr(obj, "__module__") and obj.__module__ == module.__name__
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\numpy\core\defchararray.py", line 2, in __getattr__
    from numpy._core import defchararray
  File "c:\Users\hjma9\anaconda3\envs\env_pyspark\Lib\site-packages\numpy\_core\defchararray.py", line 23, in <modu

NameError: name 'asarray' is not defined

In [ ]:
train_df.head()

### Importar Images

In [ ]:
y_train = np.array(train_df['label'])
y_val = np.array(valid_df['label'])
del train_df['label']
del valid_df['label']
x_train = train_df.values.astype(np.float32)
x_val = valid_df.values.astype(np.float32)

In [ ]:
def split_val_test(x, y, pct=0.5, shuffle=True):
    '''
    Split the dataset into validation and test sets.
    
    Parameters:
    x: numpy array of features
    y: numpy array of labels
    pct: percentage of data to use for validation (default 0.5)
    shuffle: whether to shuffle the data before splitting (default True)
    
    Returns:
    x_val, y_val, x_test, y_test: split datasets
    '''
    
    # Get the number of samples
    num_samples = len(x)
    
    # Create indices array
    indices = np.arange(num_samples)
    
    # Shuffle if requested
    if shuffle:
        np.random.seed(42)  # For reproducibility
        np.random.shuffle(indices)
    
    # Calculate split point
    split_point = int(num_samples * pct)
    
    # Split indices
    val_indices = indices[:split_point]
    test_indices = indices[split_point:]
    
    # Split the data
    x_val_split = x[val_indices]
    y_val_split = y[val_indices]
    
    x_test = x[test_indices]
    y_test = y[test_indices]
    
    return x_val_split, y_val_split, x_test, y_test

In [ ]:
# Split the validation set into new validation and test sets
x_val_new, y_val_new, x_test, y_test = split_val_test(x_val, y_val, pct=0.5, shuffle=True)

# Print the sizes of the new datasets
print(f"Original validation size: {len(x_val)}")
print(f"New validation size: {len(x_val_new)}")
print(f"Test size: {len(x_test)}")
print(f"Label distribution in new validation: {np.bincount(y_val_new)}")
print(f"Label distribution in test: {np.bincount(y_test)}")

In [ ]:
### The following

alphabet=list(string.ascii_lowercase)
alphabet.remove('j')
alphabet.remove('z')
print(len(alphabet))

### Normalise

In [ ]:
def normalise(x, method='minmax', range=(0, 1)):
    '''
    Normalize data using different methods.
    
    Parameters:
    x: numpy array of data
    method: normalization method ('minmax', 'meanstd', or 'scale255')
            - 'minmax': scale to specified range (default 0-1)
            - 'meanstd': zero mean, unit variance
            - 'scale255': simple divide by 255 (for 0-255 pixel values)
    range: tuple for min and max range when using 'minmax' method
    
    Returns:
    Normalized array
    '''
    
    if method == 'scale255':
        # Simple normalization for pixel values (0-255)
        return x / 255.0
    
    elif method == 'minmax':
        # Min-max normalization to specified range
        x_min = x.min()
        x_max = x.max()
        
        # Avoid division by zero
        if x_max - x_min == 0:
            return np.zeros_like(x)
        
        x_norm = (x - x_min) / (x_max - x_min)
        
        # Scale to desired range
        range_min, range_max = range
        return x_norm * (range_max - range_min) + range_min
    
    elif method == 'meanstd':
        # Standardization (zero mean, unit variance)
        mean = x.mean()
        std = x.std()
        
        # Avoid division by zero
        if std == 0:
            return np.zeros_like(x)
        
        return (x - mean) / std
    
    else:
        raise ValueError(f"Unknown normalization method: {method}")

In [ ]:
# Option 1: Simple normalization (0-255 pixel values)
#x_train_norm = normalise(x_train, method='scale255')

# Option 2: Min-max normalization (values between 0 and 1)
#x_train_norm = normalise(x_train, method='minmax', range=(0, 1))

# Option 3: Normalization between -1 and 1 (useful for some neural networks)
#x_train_norm = normalise(x_train, method='minmax', range=(-1, 1))

# Option 4: Standardization (mean=0, std=1)
#x_train_norm = normalise(x_train, method='meanstd')

x_train = normalise(x_train, method='scale255')

### Graficar muestras

In [ ]:
def plot_number(img, label=None):
    '''
    Plot a 28x28 image (handwritten sign).
    
    Parameters:
    img: 28x28 numpy array representing the image
    label: optional label to display in title
    '''
    plt.figure(figsize=(4, 4))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    if label is not None:
        plt.title(f'Label: {label}')
    plt.tight_layout()
    plt.show()

# Display some sample images from the training set
fig, axes = plt.subplots(3, 4, figsize=(12, 9))
fig.suptitle('Sample ASL Sign Images', fontsize=16)

for i, ax in enumerate(axes.flat):
    idx = np.random.randint(len(x_train))
    img = x_train[idx].reshape(28, 28)
    label = alphabet[y_train[idx]]
    ax.imshow(img, cmap='gray')
    ax.set_title(f'{label}')
    ax.axis('off')

plt.tight_layout()
plt.show()

### Ecuaciones para nuestro modelo


$$z^1 = W^1 X + b^1$$

$$a^1 = ReLU(z^1) $$

$$z^2 = W^2 a^1 + b^2$$

$$\hat{y} = \frac{e^{z^{2_k}}}{\sum_j{e^{z_j}}}$$


$$ \mathcal{L}(\hat{y}^{i}, y^{i}) =  - y^{i}  \ln(\hat{y}^{i}) = -\ln(\hat{y}^i)$$


$$ \mathcal{J}(w, b) =  \frac{1}{num\_samples} \sum_{i=1}^{num\_samples}-\ln(\hat{y}^{i})$$

### Funciones adicionales

#### Mini batches

In [ ]:
def create_mini_batches(x, y, batch_size=32, shuffle=True):
    '''
    Create mini batches from the dataset.
    
    Parameters:
    x: numpy array of features
    y: numpy array of labels
    batch_size: size of each mini batch (default 32)
    shuffle: whether to shuffle the data before creating batches (default True)
    
    Returns:
    List of tuples (x_batch, y_batch)
    '''
    
    num_samples = len(x)
    indices = np.arange(num_samples)
    
    # Shuffle if requested
    if shuffle:
        np.random.shuffle(indices)
    
    # Create mini batches
    mini_batches = []
    for i in range(0, num_samples, batch_size):
        batch_indices = indices[i:i + batch_size]
        x_batch = x[batch_indices]
        y_batch = y[batch_indices]
        mini_batches.append((x_batch, y_batch))
    
    return mini_batches

## Nuestra clase Linear, ReLU y Sequential

###  Clase Linear

In [ ]:
class Linear:
    '''
    Linear (fully connected) layer implementation.
    
    Parameters:
    input_size: number of input features
    output_size: number of output features (neurons)
    '''
    
    def __init__(self, input_size, output_size):
        '''
        Initialize the linear layer with weights and biases.
        
        Weights are initialized with small random values (Xavier/He initialization).
        Biases are initialized to zeros.
        '''
        # Xavier initialization for weights
        limit = np.sqrt(6 / (input_size + output_size))
        self.W = np.random.uniform(-limit, limit, size=(input_size, output_size)).astype(np.float32)
        self.b = np.zeros((1, output_size)).astype(np.float32)
        
        # Store gradients for backpropagation
        self.dW = None
        self.db = None
        
        # Store input for backpropagation
        self.X = None
    
    def forward(self, X):
        '''
        Forward pass: compute Z = X @ W + b
        
        Parameters:
        X: input data of shape (batch_size, input_size)
        
        Returns:
        Z: output of shape (batch_size, output_size)
        '''
        self.X = X
        Z = X @ self.W + self.b
        return Z
    
    def backward(self, dZ):
        '''
        Backward pass: compute gradients with respect to W, b, and X
        
        Parameters:
        dZ: gradient of loss with respect to Z, shape (batch_size, output_size)
        
        Returns:
        dX: gradient of loss with respect to X, shape (batch_size, input_size)
        '''
        batch_size = self.X.shape[0]
        
        # Gradients with respect to weights and biases
        self.dW = (self.X.T @ dZ) / batch_size
        self.db = np.sum(dZ, axis=0, keepdims=True) / batch_size
        
        # Gradient with respect to input
        dX = dZ @ self.W.T
        
        return dX
    
    def update_parameters(self, learning_rate):
        '''
        Update weights and biases using gradient descent.
        
        Parameters:
        learning_rate: learning rate for the update
        '''
        self.W -= learning_rate * self.dW
        self.b -= learning_rate * self.db

### Clase ReLU

In [ ]:
class ReLU:
    '''
    ReLU (Rectified Linear Unit) activation function implementation.
    '''
    
    def __init__(self):
        '''
        Initialize the ReLU layer.
        Store input for backpropagation.
        '''
        self.Z = None
    
    def forward(self, Z):
        '''
        Forward pass: compute ReLU activation
        ReLU(Z) = max(0, Z)
        
        Parameters:
        Z: input data of shape (batch_size, num_features)
        
        Returns:
        A: activated output of shape (batch_size, num_features)
        '''
        self.Z = Z
        A = np.maximum(0, Z)
        return A
    
    def backward(self, dA):
        '''
        Backward pass: compute gradient with respect to Z
        
        Parameters:
        dA: gradient of loss with respect to A, shape (batch_size, num_features)
        
        Returns:
        dZ: gradient of loss with respect to Z, shape (batch_size, num_features)
        '''
        # Gradient is 1 where Z > 0, and 0 where Z <= 0
        dZ = dA * (self.Z > 0).astype(np.float32)
        return dZ

### Clase Sequential

In [ ]:
class Sequential:
    '''
    Sequential model that stacks multiple layers.
    Supports forward pass, backward pass, and parameter updates.
    '''
    
    def __init__(self, layers):
        '''
        Initialize the Sequential model with a list of layers.
        
        Parameters:
        layers: list of layer objects (Linear, ReLU, etc.)
        '''
        self.layers = layers
    
    def forward(self, X):
        '''
        Forward pass through all layers.
        
        Parameters:
        X: input data of shape (batch_size, input_size)
        
        Returns:
        A: output after passing through all layers
        '''
        A = X
        for layer in self.layers:
            A = layer.forward(A)
        return A
    
    def backward(self, dZ):
        '''
        Backward pass through all layers in reverse order.
        
        Parameters:
        dZ: gradient of loss with respect to the output
        
        Returns:
        dX: gradient of loss with respect to the input
        '''
        dA = dZ
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
        return dA
    
    def update_parameters(self, learning_rate):
        '''
        Update parameters for all layers that have learnable parameters.
        
        Parameters:
        learning_rate: learning rate for the update
        '''
        for layer in self.layers:
            if hasattr(layer, 'update_parameters'):
                layer.update_parameters(learning_rate)
    
    def predict(self, X):
        '''
        Make predictions on input data.
        Returns the class with the highest probability.
        
        Parameters:
        X: input data of shape (num_samples, input_size)
        
        Returns:
        predictions: array of predicted class indices
        '''
        output = self.forward(X)
        return np.argmax(output, axis=1)

### Cost Function

In [ ]:
def softmax(Z):
    '''
    Compute softmax activation for multi-class classification.
    
    Parameters:
    Z: numpy array of shape (batch_size, num_classes)
    
    Returns:
    A: softmax probabilities of shape (batch_size, num_classes)
    '''
    # Subtract max for numerical stability
    Z_shifted = Z - np.max(Z, axis=1, keepdims=True)
    exp_Z = np.exp(Z_shifted)
    A = exp_Z / np.sum(exp_Z, axis=1, keepdims=True)
    return A


def cross_entropy_loss(y_pred, y_true):
    '''
    Compute cross-entropy loss for multi-class classification.
    
    Parameters:
    y_pred: softmax probabilities of shape (batch_size, num_classes)
    y_true: true labels (class indices) of shape (batch_size,)
    
    Returns:
    loss: average cross-entropy loss across the batch
    '''
    batch_size = len(y_true)
    # Get probabilities of true classes
    correct_logprobs = -np.log(y_pred[np.arange(batch_size), y_true])
    loss = np.mean(correct_logprobs)
    return loss


def compute_loss_and_gradients(y_pred, y_true):
    '''
    Compute loss and gradients with respect to predictions.
    
    Parameters:
    y_pred: softmax probabilities of shape (batch_size, num_classes)
    y_true: true labels (class indices) of shape (batch_size,)
    
    Returns:
    loss: cross-entropy loss
    dZ: gradient of loss with respect to pre-softmax output
    '''
    batch_size = len(y_true)
    loss = cross_entropy_loss(y_pred, y_true)
    
    # Gradient of cross-entropy with softmax
    dZ = y_pred.copy()
    dZ[np.arange(batch_size), y_true] -= 1
    
    return loss, dZ

### Loop de entrenamiento

In [ ]:
# Training loop with dynamic architecture
def train_model(model, x_train, y_train, x_val, y_val, epochs=50, batch_size=32, learning_rate=0.01):
    '''
    Train the neural network model.
    
    Parameters:
    model: Sequential model to train
    x_train: training features
    y_train: training labels
    x_val: validation features
    y_val: validation labels
    epochs: number of training epochs
    batch_size: size of mini batches
    learning_rate: learning rate for gradient descent
    
    Returns:
    history: dictionary with training metrics
    '''
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    num_classes = len(np.unique(y_train))
    
    for epoch in range(epochs):
        # Create mini batches
        mini_batches = create_mini_batches(x_train, y_train, batch_size=batch_size, shuffle=True)
        
        epoch_loss = 0
        epoch_correct = 0
        epoch_total = 0
        
        # Training phase
        for x_batch, y_batch in mini_batches:
            # Forward pass
            logits = model.forward(x_batch)
            predictions = softmax(logits)
            
            # Compute loss and gradients
            batch_loss, dZ = compute_loss_and_gradients(predictions, y_batch)
            epoch_loss += batch_loss * len(y_batch)
            
            # Count correct predictions
            batch_pred = np.argmax(predictions, axis=1)
            epoch_correct += np.sum(batch_pred == y_batch)
            epoch_total += len(y_batch)
            
            # Backward pass
            model.backward(dZ)
            
            # Update parameters
            model.update_parameters(learning_rate)
        
        # Calculate epoch metrics
        train_loss = epoch_loss / epoch_total
        train_acc = epoch_correct / epoch_total
        
        # Validation phase
        val_logits = model.forward(x_val)
        val_predictions = softmax(val_logits)
        val_loss = cross_entropy_loss(val_predictions, y_val)
        val_pred = np.argmax(val_predictions, axis=1)
        val_acc = np.sum(val_pred == y_val) / len(y_val)
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print progress
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    return history


### Create your model and train it

In [ ]:

# Define model architecture dynamically
input_size = x_train.shape[1]  # 784 for 28x28 images
num_classes = len(alphabet)  # 24 classes

# Create model with dynamic layers
model = Sequential([
    Linear(input_size, 128),
    ReLU(),
    Linear(128, 64),
    ReLU(),
    Linear(64, num_classes)
])

# Normalize validation and test data with same method
x_val_new = normalise(x_val_new, method='scale255')
x_test = normalise(x_test, method='scale255')

# Train the model
history = train_model(model, x_train, y_train, x_val_new, y_val_new, 
                      epochs=50, batch_size=32, learning_rate=0.01)

### Test your model on Random data from your test set

In [ ]:
idx = np.random.randint(len(y_test))
plot_number(x_test[idx].reshape(28,28))
pred = model.predict(x_test[idx].reshape(-1, 1))
print(f'el valor predicho es: {alphabet[pred]} el valor real es:{alphabet[y_test[idx]]}')